In [1]:
import community
import pandas as pd
from pandas import DataFrame as df
import networkx as nx
import matplotlib.pyplot as plt
import numpy as np
import random
import time
import collections

# Ignore matplotlib warnings
import warnings
from scipy.sparse import (spdiags, SparseEfficiencyWarning, csc_matrix,
    csr_matrix, isspmatrix, dok_matrix, lil_matrix, bsr_matrix)
warnings.simplefilter('ignore',SparseEfficiencyWarning)
data = pd.read_csv("karate.csv")

# Pembangkitan

In [2]:
def pembangkitan(g):
    pop_size = max(g.nodes())
    pembangkitan = []

    while len(pembangkitan) < pop_size:
        a = round(np.random.uniform(low=min(g.nodes()), high=pop_size))
        if a not in pembangkitan:
            pembangkitan.append(a)
    return pembangkitan

def adj_matrix(g, pembangkitan):
    A = nx.adjacency_matrix(g, nodelist=list(pembangkitan))
    A.setdiag(A.diagonal()*2)
    return A

# Modularitas

In [3]:
def merge(lsts):
    sets = [set(lst) for lst in lsts if lst]
    merged = True
    while merged:
        merged = False
        results = []
        while sets:
            common, rest = sets[0], sets[1:]
            sets = []
            for x in rest:
                if x.isdisjoint(common):
                    sets.append(x)
                else:
                    merged = True
                    common |= x
            results.append(common)
        sets = results
    return sets

def comm(G, pembangkitan, node):
    mrg = []
    for index, i in enumerate(pembangkitan):
        temp = []
        for j in [ u for u in G.neighbors(i)]:
            temp.append(j)
        temp.append(i)
        mrg.append(set(temp))
    mrg = list(set(map(tuple, mrg)))
    mrg.sort(key=len, reverse = True)

    final = merge(mrg)

    for i in range(len(final)):
        lc = dict()
        for j in final[i]:
            lc[j] = (G.degree(j)/(100/50))
        final[i] = lc
        
    for i in range(len(final)):
        for k,v in final[i].items():
            if node in final[i].keys():
                return i
            
def lc(G, pembangkitan, node):
    mrg = []
    for index, i in enumerate(pembangkitan):
        temp = []
        for j in [ u for u in G.neighbors(i)]:
            temp.append(j)
        temp.append(i)
        mrg.append(set(temp))
    mrg = list(set(map(tuple, mrg)))
    mrg.sort(key=len, reverse = True)

    final = merge(mrg)

    for i in range(len(final)):
        lc = dict()
        for j in final[i]:
            lc[j] = (G.degree(j)/(100/50))
        final[i] = lc
        
    for i in range(len(final)):
        for k,v in final[i].items():
            if node in final[i].keys():
                return int(sum(final[i].values()))

def Q_xi(G, pembangkitan):
    tabel = dict()
    for index, node in enumerate(pembangkitan):
        try:
            tabel[index] = [node, G.degree(node), 
                    G.number_of_edges(), 
                    lc(G, pembangkitan, node), 
                    lc(G, pembangkitan, node)*2, 
                    comm(G, pembangkitan, node),
                    (lc(G, pembangkitan, node)/G.number_of_edges())-((lc(G, pembangkitan, node)*2)/(2*G.number_of_edges()))**(2)]
        except ZeroDivisionError:
            tabel[index] = [node, G.degree(node), 
                    G.number_of_edges(), 
                    lc(G, pembangkitan, node), 
                    lc(G, pembangkitan, node)*2, 
                    comm(G, pembangkitan, node),
                    0]
        
    return tabel

def Qbest(G, pembangkitan):
    tabel = Q_xi(G, pembangkitan)
    max_com = [0]
    for i in range(len(tabel)):
        max_com.append((tabel[i])[5])

    Q = dict()    
    for i in range(max(max_com)+1):
        Qi = dict()
        for j in range(len(tabel)):
            if (tabel[j])[5] == i:
                Qi[j] = (tabel[j])[6]
        try:
            Q[i] = sum(Qi.values())/len(Qi)
        except ZeroDivisionError:
            Q[i] = 0
        
    return sum(Q.values())

# Mutasi

In [4]:
def mutasi(G, pembangkitan):
    offsprings = nx.adjacency_matrix(G, nodelist=list(pembangkitan))
    for i in range(len(offsprings.todense())):
        for j in range(len(offsprings.todense())):
            if len(offsprings.todense()) < 3:
                rand = random.randint(0,1)
            else:
                rand = random.randrange(0, len(offsprings.todense())-1, 1)
            if i != rand:
                if offsprings[i,rand] == 1:
                    offsprings[i,rand] = 0
                    offsprings[rand,i] = 0
                else:
                    offsprings[i,rand] = 1
                    offsprings[rand,i] = 1
    return offsprings

# Clean up

In [5]:
def clean_up(G, pembangkitan, A, offsprings, tabel, threshold):
    for i in range(len(A.todense())):
        for j in range(len(A.todense())):
            if i != j and offsprings[i,j] == 1:
                if offsprings[i,j] != A[i,j]:
                    if (tabel[i])[5] == (tabel[j])[5]:
                        offsprings[i,j] = 0
                        offsprings[j,i] = 0
                    else:
                        offsprings[i,j] = 1
                        offsprings[j,i] = 1

    # Menghitung community variance
    cv = dict()
    for index, node in enumerate(pembangkitan):
        cv[index] = [node, G.degree(node)]
    
    for i in range(len(offsprings.todense())):
        degree = 0
        f = 0
        for j in range(len(offsprings.todense())):
            if offsprings[i,j] == 1:
                degree = degree + 1
            if i != j and offsprings[i,j] == 1:
                if (tabel[i])[5] != (tabel[j])[5]:
                    f = f + 1
        try:
            cv[i].append(f)
            cv[i].append((cv[i])[2]/(cv[i])[1])
        except ZeroDivisionError:
            cv[i].append(0)
            cv[i].append(0)

    # memindahkan node kedalam komunitas yang dituju jika node tersebut memiliki nilai > threshold
    arr = []
    for i in range(len(offsprings.todense())):
        if (cv[i])[3] < threshold:
            arr.append(i)

    for i in range(len(arr)):
        for j in range(len(offsprings.todense())):
            if j not in arr:
                offsprings[j, arr[i]] = 0
                offsprings[arr[i], j] = 0

    return offsprings

# Crossover

In [6]:
def crossover(offsprings, prob):
    for i in range(0, len(offsprings.todense()), 2):
        for j in range(int(len(offsprings.todense())/float(100/prob))):
            try:
                temp = offsprings[i,j]
                offsprings[i,j] = offsprings[i+1,j]
                offsprings[i+1,j] = temp
            except IndexError as error:
                temp = offsprings[i,j]
                offsprings[i,j] = offsprings[i-1,j]
                offsprings[i-1,j] = temp

    # Update adjacency matrix setelah crossover
    for i in range(len(offsprings.todense())):
        for j in range(len(offsprings.todense())):
            if offsprings[i,j] != offsprings[j,i]:
                offsprings[i,j] = 1
                offsprings[j,i] = 1

    #Update adjacency matrix yang mengarah ke dirinya sendiri
    for i in range(len(offsprings.todense())):
        for j in range(len(offsprings.todense())):
            if i == j:
                offsprings[i,j] = 0
                
    return offsprings

# Make Graph from Offsprings

In [7]:
def G_offsprings(offsprings, tabel):
    OG = nx.Graph()

    for i in range(len(offsprings.todense())):
        OG.add_node((tabel[i])[0])

    arr = []
    for i in range(len(offsprings.todense())):
        for j in range(len(offsprings.todense())):
            if offsprings[i,j] == 1:
                if [(tabel[i])[0], (tabel[j])[0]] not in arr and [(tabel[j])[0], (tabel[i])[0]] not in arr:
                    arr.append([(tabel[i])[0], (tabel[j])[0]])
                    OG.add_edge((tabel[i])[0], (tabel[j])[0])

    return OG

# Update generasi

In [8]:
def update_generasi(g, pembangkitan, A, cv_parent, adj_offsprings):
    cv_offsprings = Q_xi(G_offsprings(adj_offsprings, cv_parent), pembangkitan)

    arr = []
    for i in range(len(adj_offsprings.todense())):
        if (cv_parent[i])[6] < (cv_offsprings[i])[6] or (cv_parent[i])[6] == 0 and (cv_parent[i])[6] == 0.0:
            arr = [i]
            for j in range(len(adj_offsprings.todense())):
                A[i,j] = adj_offsprings[i,j]
                A[j,i] = A[i,j]

    #Update Graph parent
    if len(arr) > 0:
        g.remove_edges_from(list(g.edges()))

        arr_g = []
        for i in range(len(adj_offsprings.todense())):
            for j in range(len(adj_offsprings.todense())):
                if adj_offsprings[i,j] == 1:
                    if [(cv_parent[i])[0], (cv_parent[j])[0]] not in arr_g and [(cv_parent[j])[0], (cv_parent[i])[0]] not in arr_g:
                        arr_g.append([(cv_parent[i])[0], (cv_parent[j])[0]])
                        g.add_edge((cv_parent[i])[0], (cv_parent[j])[0])
    return Qbest(g, pembangkitan)

# Update Community

In [9]:
def cek_community(pembangkitan, adj, cv_parent, a_offsprings, modularitas):
    cek_comm = Q_xi(G_offsprings(a_offsprings, cv_parent), pembangkitan)
    
    #check nodes and communities
    arr = dict()
    for i in range(len(pembangkitan)):
        arr[(cek_comm[i])[0]] = (cek_comm[i])[5]
    duplicate = [item for item, count in collections.Counter(arr.values()).items() if count > 1]
    delete = [k for k, v in arr.items() if v in duplicate]
    for key in delete: del arr[key]
    
    return update_community(list(arr), pembangkitan, adj, modularitas)

def update_community(PEMBANGKITAN, old_pembangkitan, adj, global_mod):
    gg = nx.Graph()
    old_graf = g.copy()
    
    for node in range(len(PEMBANGKITAN)):
        gg.add_node(PEMBANGKITAN[node])
            
    threshold = 0.8
    generasi = 1
    modularitas = [0]
    pro_cross = 80
    xbest = 0.2
    cek = []
    
#     print(len(PEMBANGKITAN))
#     print(PEMBANGKITAN)
    
    cek.append(PEMBANGKITAN)
    
    if len(PEMBANGKITAN) > 1:
        if len(PEMBANGKITAN) == 2:
            g.add_edge(PEMBANGKITAN[0],PEMBANGKITAN[1])
        elif len(PEMBANGKITAN) == 3:
            g.add_edge(PEMBANGKITAN[0],PEMBANGKITAN[1])
            g.add_edge(PEMBANGKITAN[1],PEMBANGKITAN[2])
        else:
            while len(PEMBANGKITAN) > 3:
                ADJACENCY = adj_matrix(gg, PEMBANGKITAN)
                MODULARITAS = Q_xi(gg, PEMBANGKITAN)
                MUTASI = mutasi(gg, PEMBANGKITAN)
                CLEAN_UP = clean_up(gg, PEMBANGKITAN, ADJACENCY, MUTASI, MODULARITAS, threshold)
                CROSSOVER = crossover(CLEAN_UP, pro_cross)
                CLEAN_UP_2 = clean_up(gg, PEMBANGKITAN, ADJACENCY, CROSSOVER, MODULARITAS, threshold)
                update_generasi(gg, PEMBANGKITAN, ADJACENCY, MODULARITAS, CLEAN_UP_2)
    #             print("Generasi : #",generasi)
#                 print("Q cek: ", update_generasi(gg, PEMBANGKITAN, ADJACENCY, MODULARITAS, CLEAN_UP_2))
                modularitas.append(update_generasi(gg, PEMBANGKITAN, ADJACENCY, MODULARITAS, CLEAN_UP_2))

                if len(modularitas) > 10:
                    if all(x == modularitas[-1] for x in modularitas[len(modularitas)-10:]):
                        g.add_edges_from(gg.edges())
                        break

                if max(modularitas) > xbest:
                    g.add_edges_from(gg.edges())
                    break

                generasi = generasi + 1
        generasi = 1
        mod = Q_xi(old_graf, old_pembangkitan)
        offsprings = nx.adjacency_matrix(g, nodelist=old_pembangkitan)

        PEMBANGKITAN = cek_community(old_pembangkitan, adj, mod, offsprings, global_mod)
        
        if len(cek) > 10:
            if all(x == cek[-1] for x in cek[len(cek)-10:]):
                mod = Q_xi(old_graf, old_pembangkitan)
                offsprings = nx.adjacency_matrix(g, nodelist=old_pembangkitan)
                global_mod.append(update_generasi(g, old_pembangkitan, adj, mod, offsprings))
                return 
    else:
        mod = Q_xi(old_graf, old_pembangkitan)
        offsprings = nx.adjacency_matrix(g, nodelist=old_pembangkitan)
        global_mod.append(update_generasi(g, old_pembangkitan, adj, mod, offsprings))
        return 

# Algoritma Genetika

In [10]:
g = nx.from_pandas_edgelist(data, source='source', target='target', edge_attr=None)
threshold = 0.8
generasi = 1
modularitas = [0]
pro_cross = 80
xbest = 0.6
PEMBANGKITAN = pembangkitan(g)

while max(modularitas) < xbest:
    start_time = time.time()
    ADJACENCY = adj_matrix(g, PEMBANGKITAN)
    MODULARITAS = Q_xi(g, PEMBANGKITAN)
    MUTASI = mutasi(g, PEMBANGKITAN)
    CLEAN_UP = clean_up(g, PEMBANGKITAN, ADJACENCY, MUTASI, MODULARITAS, threshold)
    CROSSOVER = crossover(CLEAN_UP, pro_cross)
    CLEAN_UP_2 = clean_up(g, PEMBANGKITAN, ADJACENCY, CROSSOVER, MODULARITAS, threshold)
    update_generasi(g, PEMBANGKITAN, ADJACENCY, MODULARITAS, CLEAN_UP_2)
    cek_community(PEMBANGKITAN, ADJACENCY, Q_xi(g, PEMBANGKITAN), CLEAN_UP_2, modularitas)
    community = dict()
    for i in g.nodes():
        community[i] = comm(g, g.nodes(), i)
        
#     dicts = dict();  
#     dicts['Generasi']   = generasi
#     dicts['Komunitas']    = max(community.values())+1
#     dicts['Modularitas']   = modularitas[-1]
#     dicts['Edges']   = g.edges()
#     hasil.append(dicts)
    
    print("==========================")
    print("Generasi : #",generasi)
    print("Komunitas :", max(community.values())+1)
    print("Q : ", modularitas[-1])
    waktu = (time.time() - start_time)
    print("%s seconds" % waktu)
    
    df=pd.DataFrame({'Generasi': generasi, 
                     'Komunitas': max(community.values())+1, 
                     'Modularitas': modularitas[-1],
                     'Times': waktu,
                     'Edges': [g.edges()]})
    df.to_csv('GA_books_10.csv', mode='a', header=False)
    
#     if len(modularitas) > 10:
#         if all(x == modularitas[-10] for x in modularitas[len(modularitas)-10:]):
#             d = dict(g.degree)
#             community = dict()
#             for i in g.nodes():
#                 color = ['#ec3939','#f2b91f','#82b440','#cd853f','#f15710','#ff00ff','#e0e0e0','#00ffff', '#f08080','#adff2f','#386665']
#                 community[i] = color[comm(g, g.nodes(), i)]
#             plt.figure(100,figsize=(20,10))
#             nx.draw_spring(g, with_labels = True, node_size=[v * 300 for v in d.values()], node_color=community.values())
#             plt.savefig('GA.svg')
#             break
        
    if max(modularitas) > xbest:
        d = dict(g.degree)
        community = dict()
        for i in g.nodes():
            color = ['#ec3939','#f2b91f','#82b440','#cd853f','#f15710','#ff00ff','#e0e0e0','#00ffff', '#f08080','#adff2f','#386665']
            community[i] = color[comm(g, g.nodes(), i)]
        plt.figure(100,figsize=(20,10))
        nx.draw_spring(g, with_labels = True, node_size=[v * 300 for v in d.values()], node_color=community.values())
        plt.savefig('GA.svg')
        break
        
    if len(modularitas) == 101:
        break
        
    generasi = generasi + 1
    


Generasi : # 1
Komunitas : 3
Q :  0.23156899810964085
2.8376917839050293 seconds
Generasi : # 2
Komunitas : 3
Q :  0.20784023668639048
1.9160981178283691 seconds
Generasi : # 3
Komunitas : 4
Q :  0.17715419501133778
2.3003110885620117 seconds
Generasi : # 4
Komunitas : 4
Q :  0.059786725715284664
1.6613059043884277 seconds
Generasi : # 5
Komunitas : 3
Q :  0.4779759183051365
1.8355329036712646 seconds
Generasi : # 6
Komunitas : 3
Q :  0.06045309777056671
1.8314251899719238 seconds
Generasi : # 7
Komunitas : 3
Q :  0.2827777777777777
1.8189170360565186 seconds
Generasi : # 8
Komunitas : 3
Q :  0.13265306122448983
1.5411767959594727 seconds
Generasi : # 9
Komunitas : 3
Q :  0.521673860011259
1.9059031009674072 seconds
Generasi : # 10
Komunitas : 3
Q :  0.521673860011259
1.3974928855895996 seconds
Generasi : # 11
Komunitas : 3
Q :  0.521673860011259
1.4284353256225586 seconds
Generasi : # 12
Komunitas : 3
Q :  0.521673860011259
1.425023078918457 seconds
Generasi : # 13
Komunitas : 3
Q :  

Generasi : # 78
Komunitas : 3
Q :  0.5200876552227903
1.4506828784942627 seconds
Generasi : # 79
Komunitas : 3
Q :  0.5200876552227903
1.4641411304473877 seconds
Generasi : # 80
Komunitas : 3
Q :  0.5200876552227903
1.5127038955688477 seconds
Generasi : # 81
Komunitas : 3
Q :  0.5200876552227903
1.4559569358825684 seconds
Generasi : # 82
Komunitas : 3
Q :  0.5200876552227903
1.448890209197998 seconds
Generasi : # 83
Komunitas : 3
Q :  0.5200876552227903
1.468101978302002 seconds
Generasi : # 84
Komunitas : 3
Q :  0.5200876552227903
1.4320671558380127 seconds
Generasi : # 85
Komunitas : 3
Q :  0.5200876552227903
1.4407050609588623 seconds
Generasi : # 86
Komunitas : 3
Q :  0.5200876552227903
1.4449348449707031 seconds
Generasi : # 87
Komunitas : 3
Q :  0.5200876552227903
1.4445722103118896 seconds
Generasi : # 88
Komunitas : 3
Q :  0.5200876552227903
1.4385871887207031 seconds
Generasi : # 89
Komunitas : 2
Q :  0.05191111111111106
1.4893720149993896 seconds
Generasi : # 90
Komunitas : 8